<a href="https://colab.research.google.com/github/vkrisvasan/llamaKV/blob/main/llama31groqneo4jKV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#https://console.groq.com/keys - register and get GROQ_API_KEY key
#https://console.groq.com/docs/libraries - Groq documentation
#https://docs.llamaindex.ai/en/stable/examples/cookbooks/GraphRAG_v1/
#https://github.com/projectwilsen/KnowledgeGraphLLM/blob/main/tutorial/2_convert_any_text_into_a_kg/notebook.ipynb
#https://docs.llamaindex.ai/en/stable/examples/cookbooks/llama3_cookbook_groq/
#https://www.youtube.com/watch?v=ky8LQE-82xs
#https://www.youtube.com/watch?v=KMXQ4SVLwmo

#https://docs.llamaindex.ai/en/stable/examples/property_graph/property_graph_neo4j/
#https://docs.llamaindex.ai/en/stable/understanding/loading/loading/
#https://console.neo4j.io/

!pip install groq
!pip install neo4j
!pip install llama-index
!pip install llama-index.core
!pip install llama-index-graph-stores-neo4j
!pip install llama-index-graph-stores-nebula
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llm-ollama

ERROR: Could not find a version that satisfies the requirement llama-index-llm-ollama (from versions: none)
ERROR: No matching distribution found for llama-index-llm-ollama


In [20]:
from groq import Groq

In [21]:
!mkdir -p 'data/receipekv1'
!wget 'https://github.com/vkrisvasan/llamaKV/blob/main/foodreceipe.txt' -O 'data/receipekv1/receipekv1.txt'
#!wget 'https://gutenberg.org/cache/epub/65061/pg65061.txt' -O 'data/receipekv1/receipekv11.txt'

--2024-08-05 02:21:46--  https://github.com/vkrisvasan/llamaKV/blob/main/foodreceipe.txt
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘data/receipekv1/receipekv1.txt’

data/receipekv1/rec     [  <=>               ] 341.11K  1.53MB/s    in 0.2s    

2024-08-05 02:21:47 (1.53 MB/s) - ‘data/receipekv1/receipekv1.txt’ saved [349296]



In [22]:

from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("./data/receipekv1/").load_data()



In [23]:
from llama_index.core import PropertyGraphIndex

In [24]:
import os
import getpass
credential_names = ["GROQ_API_KEY","NEO4J_CONNECTION_URL","NEO4J_USER","NEO4J_PASSWORD","HF_TOKEN"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

In [25]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

graph_store = Neo4jPropertyGraphStore(
    username=os.environ["NEO4J_USER"],
    password=os.environ["NEO4J_PASSWORD"],
    url=os.environ["NEO4J_CONNECTION_URL"],
)

In [26]:
!pip show llama-index

Name: llama-index
Version: 0.10.59
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse
Required-by: 


In [27]:
!pip install llama-index-llms-groq

In [28]:

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.groq import Groq

Settings.llm = Groq(model="llama-3.1-8b-instant",api_key=os.environ["GROQ_API_KEY"])

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [29]:
print(type(Settings.llm))

<class 'llama_index.llms.groq.base.Groq'>


In [30]:
# create
index = PropertyGraphIndex.from_documents(
    documents,llm=Settings.llm,
    embed_model=Settings.embed_model,
    property_graph_store=graph_store,
    show_progress=True,
    use_async=False,
)

Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting paths from text:   1%|▏         | 8/558 [00:17<30:48,  3.36s/it]WARNING:llama_index.llms.openai.utils:Retrying llama_index.llms.openai.base.OpenAI._achat in 1.901531648838449 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01j40mcr91f99avgbsge7rg2qr` on requests per minute (RPM): Limit 30, Used 30, Requested 1. Please try again in 1.305s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'requests', 'code': 'rate_limit_exceeded'}}.


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01j40mcr91f99avgbsge7rg2qr` on requests per minute (RPM): Limit 30, Used 30, Requested 1. Please try again in 1.976s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'requests', 'code': 'rate_limit_exceeded'}}

In [ ]:
!pip install nest_asyncio

In [ ]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

async def create_index():
    index = PropertyGraphIndex.from_documents(
        documents,llm=Settings.llm,
        embed_model=Settings.embed_model,
        property_graph_store=graph_store,
        show_progress=True,
        use_async=True
    )
    return index

loop = asyncio.get_event_loop()
index = loop.run_until_complete(create_index())

In [ ]:
from llama_index.core import PropertyGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
#llmkv = Groq(model="llama-3.1-8b-instant", api_key=os.environ["GROQ_API_KEY"])

index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=Settings.embed_model,
    kg_extractors=[
        SchemaLLMPathExtractor(
            llm=,
            possible_entities=["PERSON", "PLACE", "THING"],
            possible_relations=["PART_OF", "HAS", "IS_A"]
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

In [ ]:

from llama_index.core import SchemaLLMPathExtractor

entities = Literal["PERSON", "PLACE", "THING"]
relations = Literal["PART_OF", "HAS", "IS_A"]
schema = {
    "PERSON": ["PART_OF", "HAS", "IS_A"],
    "PLACE": ["PART_OF", "HAS"],
    "THING": ["IS_A"],
}

kg_extractor = SchemaLLMPathExtractor(
  llm=llm,
  possible_entities=entities,
  possible_relations=relations,
  kg_validation_schema=schema,
  strict=True,  # if false, allows values outside of spec
)

In [ ]:
!pip install youtube-transcript-api

In [ ]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(
    ytlinks=["https://www.youtube.com/watch?v=i3OYlaoj-BM"]
)

In [ ]:
def generate_content(prompt):
  reponse = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": prompt}
    ]
  )
  return reponse.choices[0].message.content

In [ ]:
output = generate_content("how to covert a pdf to a graph db")
print(output)

In [ ]:
output = generate_content("with example can you explain SOLID design principle")
print(output)